In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

In [2]:
import devanalyst.simulation.statics as S_
from devanalyst.simulation.businessObjects import UserStory, UserStoriesRepo, Ticket, TicketsRepo, WorkItem, \
UserStoryStatus, Backlog, ScrumTeam, ScrumTeamsRepo, WorkAssignments, ReleaseCycleContext
import devanalyst.simulation.simulationModels as simm

importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\simulationModels.ipynb


<h1>User Stories</h1>
<p>Capacity planning for the release: we create a backlog of stories. Initial resource allocations are made, but they might change in a sprint if needed</p>

In [13]:
class IdCounter:
    def __init__(self):
        self.counter = 1
        
    def increment(self):
        self.counter += 1
        
    def next_id(self):
        return self.counter

In [14]:
def generateNextUserStory(nextId, team, bandwidth, sprintDuration, modelsConfig):
# Returns a UserStory, randomly choosing the amount of effort the UserStory might take (while fitting withing a sprint),
# and assigning to a random developer in the team with enough time to do it, to a spec written by a randomly chosen 
# product manager from the team.
#
# Random choices are made using the random generator in the modelsConfig parameter.
#
# Depletes the time this UserStory would take from the bandwidth for the developer in question.
#
# If no developer in the team has time to do such a UserStory, returns 'None'

    estimate = modelsConfig.random.pickHowLong(sprintDuration)

    available = bandwidth[bandwidth >= estimate] # Subset of developers with enough time to do this user story
    if (len(available) == 0):
        return None
    
    developer = modelsConfig.random.pickOneIdx(available)
    bandwidth[developer] -= estimate # deplete capacity now earmarked for this user story

    productManager = modelsConfig.random.pickOne(team.productManagers)
    
    return UserStory('UserStory #' + str(nextId), estimate, developer, productManager)

In [15]:
def canResourceMoreWork(bandwidth, sprintDuration):
# Helper function used in createUserStoryBacklog.
# Returns a boolen on whether the team has enough capacity left in 'bandwidth' to resource one more user story
#
# -bandwidth: Pandas Series indexed on developers' names, with the values being the amount of unallocated days
# for that developer.
# -sprintDuration: integer representing duration of a sprint, in number of days 

    # If over half the developers in the team have at least half a sprint left, still can resource more work
    numberOfDevelopers = bandwidth.size
    condition1 = bandwidth[bandwidth > sprintDuration/2.0].size > numberOfDevelopers / 2.0
    
    # If any developer in the team has more than 150% of a sprint left, then can still resource more work
    condition2 = bandwidth.max() >= 1.5 * sprintDuration
    
    return condition1 or condition2

In [16]:
def createUserStoryBacklog(team, releaseDuration, sprintDuration, counter, modelsConfig):
#Creates user stories that a scrum team should work on for a release. Algorithm basically determines how much time 
# there is in a planned duration for a release, and based on that randomly creates user stories associated with the
# areas of responsibility of the scrum team in question, until capacity is filled to between 90% and 100%. Each user story 
# is supposed to be completed in a sprint, so a user story's estimated duration is ramdomly generated to be between
# 10% and 100% of the sprint's duration period.
#
# -team: the ScrumTeam for which the backlog is being created
# -releaseDuration: integer number of business days to the intended release date. Defaults to around 6 months
# -sprintDuration: integer number of business days that a sprint lasts. Defaults to 2 weeks. 
# -counter: an IdCounter, to tell us what is the id for the next user story to create. This method increments the counter
# -modelsConfig: a ModelsConfig instance with a random generator used in the creation of user stories.
# with each user story that is created.

    numberOfDevelopers = len(team.developers)
    initialCapacity = numberOfDevelopers *releaseDuration # number of work-days available for the release
    bandwidth = pd.Series(releaseDuration, team.developers) # initialize bandwidth to all the time left in the release

    #numberOfSprints = releaseDuration/sprintDuration
    stories = []
    backlog = Backlog()     
    
    # We allocate as much of the team as practical, with user stories ranging in estimated duration from 1 day
    # to the number of days in a sprint. To avoid artificial situations, don't aim to fill every single day,
    # so will stop when we have allocated at least 10% and there is no developer left with enough bandwidth
    # to resource a medium-size user story.
    # To avoid infinite loops, we force a stop after 1000 tries, though most likely we will exit well before then.
    next_id = counter.next_id()
    while (canResourceMoreWork(bandwidth, sprintDuration) and next_id < 10000): #To avoid infinite loops, cap number of user stories
        story = generateNextUserStory(next_id, team, bandwidth, sprintDuration, modelsConfig) #this call mutates bandwidth
        if (story == None):
            continue #Try again, maybe we tried for a very large estimate but a smaller user story can still be resourced
        stories.append(story)
        
        counter.increment()
        next_id = counter.next_id()
        backlog.pendingUserStories.append(UserStoryStatus(story.userStoryId))       
    
    return stories, backlog

<h1>Do a sprint</h1>

In [18]:
#def chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, sprintDuration, modelsConfig, sprint):
def chooseWhatToDoInSprint(modelsConfig):
# At the start of a sprint, elects what items a team will do in a sprint
#
# -teamId: string identifier of team for which we are choosing work for this sprint
# sprintDuration: length of the sprint for which work needs to be chosen, in man-days
# -modelsConfig: a ModelsConfig instance with parameters for making the choice of work to do in the sprint
# -sprint: integer representing which sprint we are in

    
    allocationModel = modelsConfig.allocationModel
    work = WorkAssignments(modelsConfig.context)
    
    return allocationModel.allocate(work, modelsConfig) # Mutates work

In [19]:
def inflowOfTickets(modelsConfig):
# Returns an array of newly generated tickets against the work done by the team
    # TODO - NOT YET IMPLEMENTED
    return []

In [20]:
def bookDeveloperEffort(items, budget, costModels):
# Helper class, used as part of the book keeping involved in delivering a sprint.
#
# Returns what portion of the budget, if any, was left over after booking the work done by the developer
#
# -items: a list of WorkItem objects that record work that should be done in a sprint by a single developer. 
# It will be mutated by this fundtion by recording what were the actual number of man-days spent on each WorkItem, and to what degree it was completed
# -budget: number of man-days that the developer was given to try to complete the WorkItems
# -costModels: an array of CostModel instances

    for task in items:
        if budget <= 0:
            break # No more tasks progressed during this sprint
        realCost = simm.computeRealCost(task, costModels)
        if realCost == 0:
            continue # Boundary base. Shouldn't happen, but if someone entered an estimate of 0 don't want to divide by 0
        if realCost <= budget:
            task.percentAchieved = 1
            task.actual = realCost
            budget -= realCost
        else:
            # Can only complete part of the item
            task.percentAchieved = budget/realCost
            task.actual = budget
            budget = 0
    return budget

In [21]:
#def deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, sprintDuration, modelsConfig):
def deliverSprint(work, modelsConfig):
# Based on the team's productivy during the sprint, record how much of the work initially planned for the sprint
# actually gets done
#
# -work: a WorkAssignment as it was at the start of the sprint. It is mutated by this method by recording, for each
# WorkItem in the WorkAssignment, what percentage of it got accomplished
# -sprintDuration: number of man-days that sprint lasted
# -modelsConfig: a ModelsConfig instance to model how real costs differ from estimates

    teamId                 = modelsConfig.context.teamId
    teamsRepo              = modelsConfig.context.teamsRepo
    storiesRepo            = modelsConfig.context.storiesRepo
    ticketsRepo            = modelsConfig.context.ticketsRepo
    sprintDuration         = modelsConfig.context.sprintDuration
    sprint                 = modelsConfig.context.sprint

    team = teamsRepo.findTeam(teamId)
    for person in team.developers:
        budget = sprintDuration # Developer has up to these many days to complete work in the sprint
        
        # First, deliver the work from the current sprint
        budget = bookDeveloperEffort(work.getWorkItems(person, S_.CURRENT_SPRINT), budget, modelsConfig.costModels)
                
        # Second, developer still has some time in his/her budget for this sprint, then he/she might have started to work
        # on things for the next sprint, in which case record that progress
        budget = bookDeveloperEffort(work.getWorkItems(person, S_.NEXT_SPRINT), budget, modelsConfig.costModels)    

In [1]:
#def updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, newTickets, sprint):
def updateBacklogAfterSprint(work, modelsConfig):
# At the end of a sprint, updates the backlog of a team based on the work completed in this sprint
#
# -work: a WorkAssignment, which reflects what was accomplished during the sprint through the field percentAchieved
# in each of the WorkItems in the various arrays within the work.allocations dictionary

    teamId                 = modelsConfig.context.teamId
    teamsRepo              = modelsConfig.context.teamsRepo

    team = teamsRepo.findTeam(teamId)
    completedWork = []
    for person in team.developers:
        completedWork.extend(work.getWorkItems(person, S_.CURRENT_SPRINT))
        completedWork.extend(work.getWorkItems(person, S_.NEXT_SPRINT))
    
    team.backlog.updateStatus(completedWork, modelsConfig.context) 

<h1>Functions to Generate Timecards</h1>

In [23]:
def addBusinessDays(start, duration):
# Returns a datetime.date object which is several business days after the input 'start'
    remainsToAdd = duration
    current = start
    while remainsToAdd > 0:
        current = current + timedelta(1)
        weekday = current.weekday()
        if weekday >= 5: # sunday = 6
            continue
        remainsToAdd -= 1
    return current

def subtractBusinessDays(start, duration):
# Returns a datetime.date object which is several business days before the input 'start'
    remainsToSubtract = duration
    current = start
    while remainsToSubtract > 0:
        current = current - timedelta(1)
        weekday = current.weekday()
        if weekday >= 5: # sunday = 6
            continue
        remainsToSubtract -= 1
    return current

In [24]:
class ReleaseLog:
    
    BREAKOUT = 'Breakout'
    COMPLETED = 'Completed'
    NEW_WORK = 'New Work'
    NOT_STARTED = 'Not Started'
    OVER_BUDGET = 'Over Budget'
    PRIOR_PROGRESSED = 'Prior Progressed'
    PRIOR_TO_FINISH = 'Prior to Finish'
    PROGRESSED = 'Progressed'
    TO_FINISH = 'To Finish'
    UNPLANNED = 'Unplanned'
    
    SNAPSHOTS = ['planned_Start_CURRENT_SPRINT', 'planned_End_CURRENT_SPRINT', 
                      'planned_Start_NEXT_SPRINT', 'planned_End_NEXT_SPRINT', 
                      'backlog', 'Outcome']  
    
    def __init__(self):
        self.log = {}
        
    
    def _getPlans(self, work):
        unassigned = work.allocations[S_.UNPLANNED][S_.OWNER_TBD]
        unplanned = []
        unplanned.extend(unassigned[S_.BUGS_ON_COMPLETED_STORIES])
        unplanned.extend(unassigned[S_.BUGS_ON_UNFINISHED_STORIES])
        unplanned.extend(unassigned[S_.UNFINISHED_STORIES])
    
        planned_CURRENT_SPRINT = []
        subwork = work.allocations[S_.CURRENT_SPRINT]
        for person in subwork.keys():
            planned_CURRENT_SPRINT.extend(subwork[person][S_.BUGS_ON_COMPLETED_STORIES])
            planned_CURRENT_SPRINT.extend(subwork[person][S_.BUGS_ON_UNFINISHED_STORIES])
            planned_CURRENT_SPRINT.extend(subwork[person][S_.UNFINISHED_STORIES])

        planned_NEXT_SPRINT = []
        subwork = work.allocations[S_.NEXT_SPRINT]
        for person in subwork.keys():
            planned_NEXT_SPRINT.extend(subwork[person][S_.BUGS_ON_COMPLETED_STORIES])
            planned_NEXT_SPRINT.extend(subwork[person][S_.BUGS_ON_UNFINISHED_STORIES])
            planned_NEXT_SPRINT.extend(subwork[person][S_.UNFINISHED_STORIES])
            
        return planned_CURRENT_SPRINT, planned_NEXT_SPRINT, unplanned
        
    def snapshotSprintBacklog(self, modelsConfig):

        teamId                 = modelsConfig.context.teamId
        teamsRepo              = modelsConfig.context.teamsRepo
        storiesRepo            = modelsConfig.context.storiesRepo
        ticketsRepo            = modelsConfig.context.ticketsRepo
        sprintDuration         = modelsConfig.context.sprintDuration
        sprint                 = modelsConfig.context.sprint

        if (teamId not in self.log.keys()):
            self.log[teamId] = {}
        if (sprint not in self.log[teamId].keys()):
            self.log[teamId][sprint] = {}
    
        self.log[teamId][sprint]['backlog']                              = ReleaseLog._build_backlog_df(teamId,
                                                                                                     teamsRepo,
                                                                                                     storiesRepo,
                                                                                                     sprintDuration)
    def snapshotSprintPlan(self, work, modelsConfig):

        teamId                 = modelsConfig.context.teamId
        teamsRepo              = modelsConfig.context.teamsRepo
        storiesRepo            = modelsConfig.context.storiesRepo
        ticketsRepo            = modelsConfig.context.ticketsRepo
        sprintDuration         = modelsConfig.context.sprintDuration
        sprint                 = modelsConfig.context.sprint

        if (teamId not in self.log.keys()):
            self.log[teamId] = {}
        if (sprint not in self.log[teamId].keys()):
            self.log[teamId][sprint] = {}
    
        planned_CURRENT_SPRINT, planned_NEXT_SPRINT, unplanned = self._getPlans(work)
        
        phase = 'Start'
        self.log[teamId][sprint]['planned_' + phase + '_CURRENT_SPRINT'] = ReleaseLog._build_plan_df(planned_CURRENT_SPRINT,
                                                                                                     teamsRepo,
                                                                                                     storiesRepo,
                                                                                                     sprintDuration)
        self.log[teamId][sprint]['planned_' + phase + '_NEXT_SPRINT']    = ReleaseLog._build_plan_df(planned_NEXT_SPRINT,
                                                                                                     teamsRepo,
                                                                                                     storiesRepo,
                                                                                                     sprintDuration)
        
    def snapshotSprintOutcome(self, work, modelsConfig):

        teamId                 = modelsConfig.context.teamId
        teamsRepo              = modelsConfig.context.teamsRepo
        storiesRepo            = modelsConfig.context.storiesRepo
        ticketsRepo            = modelsConfig.context.ticketsRepo
        sprintDuration         = modelsConfig.context.sprintDuration
        sprint                 = modelsConfig.context.sprint

        if (teamId not in self.log.keys()):
            self.log[teamId] = {}
        if (sprint not in self.log[teamId].keys()):
            self.log[teamId][sprint] = {}
    
        planned_CURRENT_SPRINT, planned_NEXT_SPRINT, unplanned = self._getPlans(work)
        
        phase = 'End'
        self.log[teamId][sprint]['planned_' + phase + '_CURRENT_SPRINT'] = ReleaseLog._build_outcome_df(planned_CURRENT_SPRINT,
                                                                                                     teamsRepo,
                                                                                                     storiesRepo,
                                                                                                     sprintDuration)
        self.log[teamId][sprint]['planned_' + phase + '_NEXT_SPRINT']    = ReleaseLog._build_outcome_df(planned_NEXT_SPRINT,
                                                                                                     teamsRepo,
                                                                                                     storiesRepo,
                                                                                                     sprintDuration)
        self.log[teamId][sprint]['Outcome'] = work.committedTasks()
        
    def _getCategory(estimate):
        cat = int(estimate)
        return cat
        
    def _build_plan_df(allocations, teamsRepo, storiesRepo, sprintDuration):
        cols = [ReleaseLog.BREAKOUT]
        categories = []
        # Used categories go 1, 2, ..., sprintDuration, corresponding to the original estimates for user stories. 
        # Spurious category 0 will be never used, but no harm in having it.
        for x in range(sprintDuration + 1): 
            
            category = x
            categories.append(category)
        cols.extend(categories)

        prior_progressed = [ReleaseLog.PRIOR_PROGRESSED]
        prior_remaining = [ReleaseLog.PRIOR_TO_FINISH]
        new_work = [ReleaseLog.NEW_WORK] 
        for cat in categories:
            prior_progressed.append(0.0)
            prior_remaining.append(0.0)
            new_work.append(0.0)
        
        for item in allocations:
            story = storiesRepo.findStory(item.userStoryId)
            uss = teamsRepo.getUserStoryStatus(item.userStoryId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
        
            if (uss.percentAchieved == 0.0):
                new_work[idx] += 1
            else:
                if (uss.percentAchieved < 1.0):
                    prior_progressed[idx] += (story.originalEstimate * uss.percentAchieved)/story.originalEstimate
                    prior_remaining[idx] += (story.originalEstimate * (1-uss.percentAchieved))/story.originalEstimate
         
        data = []
        data.append(prior_progressed)
        data.append(prior_remaining)
        data.append(new_work)
        data_df = pd.DataFrame(data, columns=cols)
        return data_df
    
    def _build_outcome_df(allocations, teamsRepo, storiesRepo, sprintDuration):
        cols = [ReleaseLog.BREAKOUT]
        categories = []
        for x in range(sprintDuration + 1): # inlude 0 as a category for tasks with less than 1 person-day to go
            category = x
            categories.append(category)
        cols.extend(categories)

        completed = [ReleaseLog.COMPLETED]
        progressed = [ReleaseLog.PROGRESSED]
        to_finish = [ReleaseLog.TO_FINISH] 
        not_started = [ReleaseLog.NOT_STARTED] 
        over_budget = [ReleaseLog.OVER_BUDGET] 
        for cat in categories:
            completed.append(0.0)
            progressed.append(0.0)
            to_finish.append(0.0)
            not_started.append(0.0)
            over_budget.append(0.0)
        
        for item in allocations:
            story = storiesRepo.findStory(item.userStoryId)
            uss = teamsRepo.getUserStoryStatus(item.userStoryId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
        
            if (uss.percentAchieved == 1.0):
                completed[idx] += 1
            else:
                if (uss.percentAchieved == 0.0):
                    not_started[idx] += 1
                else:
                    progressed[idx] += (story.originalEstimate * uss.percentAchieved)/story.originalEstimate
                    to_finish[idx] += (story.originalEstimate * (1-uss.percentAchieved))/story.originalEstimate
                    
            over_budget[idx] += (item.actual - item.estimate*item.percentAchieved)/story.originalEstimate
         
        data = []
        data.append(completed)
        data.append(progressed)
        data.append(to_finish)
        data.append(not_started)
        data.append(over_budget)
        data_df = pd.DataFrame(data, columns=cols)
        return data_df

    def _build_backlog_df(teamId, teamsRepo, storiesRepo, sprintDuration):
        
        team = teamsRepo.findTeam(teamId)
        backlog = team.backlog.pendingUserStories
        
        cols = [ReleaseLog.BREAKOUT]
        categories = []
        for x in range(sprintDuration + 1): # inlude 0 as a category for tasks with less than 1 person-day to go
            category = x
            categories.append(category)
        cols.extend(categories)

        unplanned = [ReleaseLog.UNPLANNED]
        completed = [ReleaseLog.COMPLETED]
        planned_progressed = [ReleaseLog.PROGRESSED]
        planned_remaining = [ReleaseLog.TO_FINISH]
        not_started = [ReleaseLog.NOT_STARTED]

        for cat in categories:
            unplanned.append(0.0)
            completed.append(0.0)
            planned_progressed.append(0.0)
            planned_remaining.append(0.0)
            not_started.append(0.0)
        
        for uss in backlog:
            story = storiesRepo.findStory(uss.userStoryId)
            cat = ReleaseLog._getCategory(story.originalEstimate)
            idx = categories.index(cat) + 1 # offset by 1 since first element in arrays is the layer
        
            if (uss.percentAchieved == 0.0):
                if (uss.planned == False):
                    unplanned[idx] += 1
                else:
                    not_started[idx] += 1
            else:
                if (uss.percentAchieved == 1.0):
                    completed[idx] += 1                    
                else:
                    planned_progressed[idx] += (story.originalEstimate * uss.percentAchieved)/story.originalEstimate
                    planned_remaining[idx] += (story.originalEstimate * (1-uss.percentAchieved))/story.originalEstimate
         
        data = []
        data.append(unplanned)
        data.append(completed)
        data.append(planned_progressed)
        data.append(planned_remaining)
        data.append(not_started)
        data_df = pd.DataFrame(data, columns=cols)
        return data_df
    
    # Returns a dataframe of merged dataframes across all sprints and teams for a given snaphot name.
    # -snapshot_name: a string for the type of snapshot we will merge across all sprints and teams.
    # Possible choices:
    #   'planned_Start_CURRENT_SPRINT'
    #   'planned_End_CURRENT_SPRINT'
    #   'planned_Start_NEXT_SPRINT'
    #   'planned_End_NEXT_SPRINT'
    #   'backlog'
    #   'Outcome'    
    def mergeLogs(self, snapshot_name):
        log = self.log
        frames = []
        for teamId in log.keys():
            for sprint in log[teamId].keys():
                snapshot_df = log[teamId][sprint][snapshot_name]
                snapshot_df['Team Id'] = teamId
                snapshot_df['Sprint'] = sprint
                frames.append(snapshot_df)
        merged_df = pd.concat(frames)
        merged_df = merged_df.reset_index().drop(columns=['index'])
        return merged_df 

In [2]:
def runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, startDate, sprintDuration, numberOfSprints, modelsConfig):
# Runs the dynamics of release cycle, iterating through all the sprints. For each sprint a WorkAssignment work sheet 
# is created for each team, recording what the team aims to do during the sprint. At the end of the sprint the 
# work sheet is updated with what was actually accomplished.
#
# This function returns two dataframes: a detailed timecards dataframe (one row for each row in any of the sheets, across all 
# teams and all sprints), and a more aggregated dataframe recording the work sheet object (a WorkAssignment) for each sprint
# and team.
# The timecards dataframes has only scalars. The work sheet dataframe has WorkAssignment objects in one column, and an
# informative list of backlog items not yet planned.
#
# Input parameters:
# -teamsRepo: a ScrumTeamsRepo containing all the scrum teams that work full time in this release. It is presumed to have
# been given a backlog of user stories to complete in this release.
# -ticketsRepo: a TicketsRepo, presumed empty at the start of the release, where tickets are stored as they are reported 
# throughout the release.
# -storiesRepo: a UserStoriesRepo containing the UserStory's that are referenced in by the teams' backlogs.
# -startDate: a datetime object for when the release cycle starts
# -sprintDuration: an integer representing the number of man-days that a sprint lasts.
# -numberOfSprints: an integer, representing for how many sprints are we going to run the release. This should be set
# high enough that all work (including delays and bug fixing) is completed within these numberOfSprints, even if the 
# original estimates are for much less. The reason for needing to set it high enough is that timecard entries will stop
# after numberOfSprints.
# -modelsConfig: a ModelsConfig instance. Has the simulation models and parameters used to drive the simulated release
# cycle.
    
    timecards_dict = {'Sprint': [], 'Date': [], 'Team': [],'Developer': [], \
                      'User Story': [], 'Task Type': [], 'Time Spent': []}
    
    #Used to record how decisions are made at each sprint.
    log = ReleaseLog()
    
    sprintEndDate = startDate
    for n in range(numberOfSprints):
        sprintEndDate = addBusinessDays(sprintEndDate, sprintDuration)
        sprint = n+1
        for team in teamsRepo.teams:
            teamId = team.teamId

            # Set context to inform the environment in which decisions and events for this cycle occur
            modelsConfig.context = ReleaseCycleContext(teamId, teamsRepo, storiesRepo, ticketsRepo, sprint, sprintDuration)
            
            log.snapshotSprintBacklog(modelsConfig) 

            # Do the sprint
            work = chooseWhatToDoInSprint(modelsConfig) 
            log.snapshotSprintPlan(work, modelsConfig) 

            deliverSprint(work, modelsConfig) # mutates 'work'
            inflow = inflowOfTickets(modelsConfig)
            updateBacklogAfterSprint(work, modelsConfig) # Does not mutate 'work'
            
            log.snapshotSprintOutcome(work, modelsConfig) 


            sprintOutcome = work.committedTasks()
            for index, row in sprintOutcome.iterrows():
                timecards_dict['Sprint'].append(n+1)
                timecards_dict['Date'].append(sprintEndDate)
                timecards_dict['Developer'].append(row['Owner'])
                timecards_dict['Team'].append(teamId)
                timecards_dict['User Story'].append(row['User Story Id'])
                timecards_dict['Task Type'].append(row['Task Type'])
                timecards_dict['Time Spent'].append(row['Effort Spent']) #Even 0, record it to indicate it was in scope
            # Boundary case: if we finished all work before all sprints are through, enter 0 time spent
            if (sprintOutcome.index.size == 0):
                timecards_dict['Sprint'].append(n+1)
                timecards_dict['Date'].append(sprintEndDate)
                timecards_dict['Developer'].append(None)
                timecards_dict['Team'].append(teamId)
                timecards_dict['User Story'].append(None)
                timecards_dict['Task Type'].append(None)
                timecards_dict['Time Spent'].append(0) 
                        
    return pd.DataFrame(timecards_dict), log

In [26]:
# Number of user stories worked on in a Sprint, either to implement or fix bugs on them
def countUniques(seriesGroup):
    if (seriesGroup.unique()[0] == None):
        return 0
    else:
        return seriesGroup.unique().size

def releaseBurnout(timecard):
    bystory = timecard.groupby('User Story')
    ends = bystory['Sprint'].max()
    starts = bystory['Sprint'].min()

    bysprint = timecard.groupby('Sprint')


    #counts = bysprint['User Story'].apply(lambda x: if x.unique()[0]!= None: len(x.unique()) else: 0) 

    counts = bysprint['User Story'].apply(lambda x: countUniques(x)) 


    efforts = bysprint['Time Spent'].sum()

    d = {'Sprint': [], 'Stories Started': [], 'Stories Progressed': [], 'Stories Completed': [], 'Effort': [],\
    'Active Developers': []}

    # Initialize
    for sprint in counts.index:
        d['Sprint'].append(sprint)
        d['Stories Progressed'].append(counts[sprint])
        d['Effort'].append(efforts[sprint])

        for s,group in bysprint:
            if s==sprint: #found it
                count = group['Developer'].unique().size
                if (count == 1): #It might be that actually no developer has work left, if the 'unique' is None
                    if (None == (group['Developer'].unique())[0]):
                        count = 0
                d['Active Developers'].append(count)

        # Allocate space, we'll come back to set a value
        d['Stories Completed'].append(0)
        d['Stories Started'].append(0) 

    for story in ends.index:
        sprint = ends[story]
        if sprint in d['Sprint']:
            idx = d['Sprint'].index(sprint)
            d['Stories Completed'][idx] += 1
        else:
            d['Sprint'].append(sprint)
            d['Stories Completed'].append(1)
            d['Stories Started'].append(0) # Allocate space, we'll come back to set a value
    for story in starts.index:
        sprint = starts[story]
        if sprint in d['Sprint']:
            idx = d['Sprint'].index(sprint)
            d['Stories Started'][idx] += 1
        else:
            d['Sprint'].append(sprint)
            d['Stories Started'].append(1)
        
        
    df = pd.DataFrame(d)
    return df.sort_values(by='Sprint')